In [150]:
import pandas as pd

In [151]:
df = pd.read_csv('Preprocessing1.csv')

In [152]:
df.head()

,Airline,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price,Month,WeekDay,Day
0,Jet Airways,Delhi,Banglore,DEL → BOM → COK,20:00,04:25 10 Jun,26h 35m,1 stop,In-flight meal not included,14924,6,Thursday,6.0
1,Jet Airways,Delhi,Cochin,DEL → BOM → COK,16:00,19:00 10 Jun,27h,1 stop,In-flight meal not included,10577,6,Sunday,9.0
2,Jet Airways,Mumbai,Hyderabad,BOM → HYD,19:35,21:05,1h 30m,non-stop,No info,5678,3,Friday,15.0
3,Multiple carriers,Delhi,Banglore,DEL → BOM → COK,18:55,01:30 16 Jun,15h 10m,1 stop,In-flight meal not included,7408,5,Monday,6.0
4,Air India,Delhi,Cochin,DEL → COK,17:10,17:55,8h 20m,non-stop,No info,6724,6,Monday,24.0


In [153]:
df["WeekDay"].unique()

array(['Thursday', 'Sunday', 'Friday', 'Monday', 'Saturday', 'Wednesday',
       'Tuesday'], dtype=object)

Average Ticket Price

In [154]:
df["Price"].mean()

9027.895555555555

Month with highest number of flights

In [155]:
df["Month"].mode()

0    5
Name: Month, dtype: int64

Average price on Weekends vs Weekdays

In [156]:
df_weekends = df[df["WeekDay"].isin(("Saturday", "Sunday"))]
df_weekdays = df[df["WeekDay"].isin(("Monday", "Tuesday", "Wednesday", "Thursday", "Friday"))]

avg_price_weekends = df_weekends["Price"].mean()
avg_price_weekdays = df_weekdays["Price"].mean()
print(avg_price_weekends, avg_price_weekdays, "Greater price on", ("Weekends" if avg_price_weekends > avg_price_weekdays else "Weekdays"))

df_weekends = []
df_weekdays = []

9058.016077170418 9015.219666215608 Greater price on Weekends


Replace No Info with No info

In [157]:
df["Additional_Info"].replace("No Info", "No info", inplace = True)
df["Additional_Info"]

0       In-flight meal not included
1       In-flight meal not included
2                           No info
3       In-flight meal not included
4                           No info
                   ...             
9445    In-flight meal not included
9446                        No info
9447                        No info
9448                        No info
9449                        No info
Name: Additional_Info, Length: 9450, dtype: object

In [158]:
len(df[(df["Airline"] == "IndiGo") & (df["Additional_Info"] == "No info")])

1650

Change Durations to Seconds and then find Average Duration

My dumb manual approach

In [159]:
durations = df["Duration"]
durations_split = [duration.split(' ') for duration in durations]
durations_in_s = []

for duration in durations_split:
    # There are a few possible cases
    # one is that something like ['1h', '2m'] exist
    # second is ['1h']
    # third is ['2m']
    # I need to check for all 3
    # If length is 2, go forward with normal approach
    # If length is 1, check if last letter is 'h' or 'm'
    # Then do the needful

    if len(duration) == 2:
        hours_s, minutes_s = int(duration[0][:-1]) * 3600, int(duration[1][:-1]) * 60
        durations_in_s.append(hours_s + minutes_s)
    else:
        time = duration[0]
        if time[-1] == 'h':
            # Only hours
            hours_s = int(time[:-1]) * 3600
            durations_in_s.append(hours_s)
        else:
            minutes_s = int(time[:-1]) * 60
            durations_in_s.append(minutes_s)

df["Duration_s"] = pd.Series(durations_in_s)

In [160]:
df["Duration_s"].mean()

38957.93650793651

Pandas Clean Approach

In [161]:
df_copy = df.copy()

In [162]:
df_copy['Duration'] = pd.to_timedelta(df_copy['Duration'])
df_copy['Duration'].dt.seconds
df_copy['Duration'].mean()

Timedelta('0 days 10:49:17.936507936')

In [163]:
df_copy['Duration_s'] = pd.Series(durations_in_s)
df_copy['Duration_s'].mean()

38957.93650793651

In [164]:
!python --version

Python 3.11.5


Convert dep_time and arrival_time to take only their hour component and then use it to categorize into 4 main types 'Morning', 'Afternoon', 'Evening', 'Night'

In [165]:
def categorize_hour(hour_min: str) -> str:
    hour = int(hour_min.split(':')[0])
    match hour:
        case hour if 5 <= hour < 12:
            return "Morning"
        case hour if 12 <= hour < 17:
            return "Afternoon"
        case hour if 17 <= hour < 20:
            return "Evening"
        case hour if 20 <= hour or hour < 5:
            return "Night"
        case _:
            print(hour)
            raise ValueError("Wrong input value tehe")

df_copy['Dep_Time'] = df_copy['Dep_Time'].apply(categorize_hour)
df_copy['Arrival_Time'] = df_copy['Arrival_Time'].apply(categorize_hour)
df_copy['Dep_Time'], df_copy['Arrival_Time']

(0           Night
 1       Afternoon
 2         Evening
 3         Evening
 4         Evening
           ...    
 9445      Evening
 9446      Evening
 9447      Morning
 9448      Morning
 9449    Afternoon
 Name: Dep_Time, Length: 9450, dtype: object,
 0         Night
 1       Evening
 2         Night
 3         Night
 4       Evening
          ...   
 9445    Evening
 9446      Night
 9447    Evening
 9448    Evening
 9449      Night
 Name: Arrival_Time, Length: 9450, dtype: object)

In [166]:
len(df_copy[(df_copy['Dep_Time'] == 'Morning') & (df_copy['Arrival_Time'] == 'Evening')])

922

Encode Weekends as 1 and Weekdays as 0

In [167]:
def encode_day(day: str) -> str:
    if day in ("Monday", "Tuesday", "Wednesday", "Thursday", "Friday"):
        return 0
    else:
        return 1
    
df_copy['WeekDay'] = df_copy['WeekDay'].apply(encode_day)
df_copy['WeekDay']

0       0
1       1
2       0
3       0
4       0
       ..
9445    0
9446    1
9447    1
9448    1
9449    0
Name: WeekDay, Length: 9450, dtype: int64

In [170]:
df_copy['WeekDay'].value_counts()

WeekDay
0    6651
1    2799
Name: count, dtype: int64